In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive/hankuk"
!cd "/content/gdrive/My Drive/hankuk"

mHealth_subject10.csv  mHealth_subject4.csv  mHealth_subject8.csv
mHealth_subject1.csv   mHealth_subject5.csv  mHealth_subject9.csv
mHealth_subject2.csv   mHealth_subject6.csv
mHealth_subject3.csv   mHealth_subject7.csv


In [ ]:
!pip install tensorflow_federated

In [ ]:
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import pandas as pd
from collections import OrderedDict
import csv
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


b'Hello, World!'

In [ ]:
def get_data(path):
  train_sensors = []
  train_labels = []

  with open(path) as train:
    csv_reader = csv.reader(train)
    
    for row in csv_reader:

      sensor = row[0:-1]
      sensor_list = np.array(sensor, dtype='float')
      train_sensors.append(sensor_list)

      label_str = row[-1]
      label = int(label_str)
      train_labels.append(label)

  return train_sensors, train_labels



In [ ]:
x_train_1, y_train_1 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject1.csv')
x_train_2, y_train_2 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject2.csv')
x_train_3, y_train_3 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject3.csv')
x_train_4, y_train_4 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject4.csv')
x_train_5, y_train_5 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject5.csv')
x_train_6, y_train_6 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject6.csv')
x_train_7, y_train_7 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject7.csv')
x_train_8, y_train_8 = get_data('/content/gdrive/My Drive/hankuk/mHealth_subject8.csv')

In [ ]:
def create_federated_data(x_train, y_train):

    orderDict = OrderedDict()
    sensors_list = []
    
    x_train = x_train.reshape(len(x_train), 21, 1)
    orderDict['x'] = np.array(x_train)
    orderDict['y'] = np.array(y_train)
    dataset = tf.data.Dataset.from_tensor_slices(orderDict)

    return dataset

federated_data_client_1 = create_federated_data(np.array(x_train_1), np.array(y_train_1)) 
federated_data_client_2 = create_federated_data(np.array(x_train_2), np.array(y_train_2)) 
federated_data_client_3 = create_federated_data(np.array(x_train_3), np.array(y_train_3)) 
federated_data_client_4 = create_federated_data(np.array(x_train_4), np.array(y_train_4)) 
federated_data_client_5 = create_federated_data(np.array(x_train_5), np.array(y_train_5)) 
federated_data_client_6 = create_federated_data(np.array(x_train_6), np.array(y_train_6)) 
federated_data_client_7 = create_federated_data(np.array(x_train_7), np.array(y_train_7)) 
federated_data_client_8 = create_federated_data(np.array(x_train_8), np.array(y_train_8)) 

In [ ]:
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['x'], [-1, 21]),
        y=tf.reshape(element['y'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


In [ ]:
preprocessed_example_dataset = preprocess(federated_data_client_1)

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(21,)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(7, activation='softmax'),
  ])

In [ ]:
dummy_batch = tf.nest.map_structure(lambda x: x.numpy(), iter(federated_data_client_1).next())
print(dummy_batch['x'].shape)

(21, 1)


In [ ]:
def model_fn():
  # create a new model to call within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[21,6],float32[6]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = iterative_process.initialize()

In [ ]:
state, metrics = iterative_process.next(state, federated_data_client_1)
print('round  1, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_2)
print('round  2, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_3)
print('round  3, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_4)
print('round  4, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_5)
print('round  5, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_6)
print('round  6, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_7)
print('round  7, metrics={}'.format(metrics))
state, metrics = iterative_process.next(state, federated_data_client_8)
print('round  8, metrics={}'.format(metrics))

RuntimeError: ignored